In [227]:
from __future__ import division
from __future__ import print_function
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
import pandas as pd
import numpy  as np

In [20]:
data_dir = "./small-10-datasets/"
data_titles = ['accidents', 'baudio', 'bnetflix', 'dna', 'jester', 'kdd', 'msnbc',
              'nltcs', 'plants', 'r52']
test  = dict()
train = dict()
valid = dict()

for title in data_titles:
    test[title]  = np.loadtxt(data_dir + title + '.test.data', delimiter=',')
    train[title] = np.loadtxt(data_dir + title + '.ts.data', delimiter=',')
    valid[title] = np.loadtxt(data_dir + title + '.valid.data', delimiter=',')

In [102]:
# Bayesian Network, No edges Algorithm
# Takes in a dataset of binary variables and takes a test set of binary variables.
def BN_NE(T, t):
    cols = np.transpose(T)
    n    = len(cols)
    return np.array([(sum(cols[i] == t[i]) / len(cols[i])) for i in range(0, n)])

class BN:
    def __init__():
        
    def fit():
        
    # Find Weights
    def FW(T):
        n = len(T)
        A = np.transpose(T)
        matrix = [[0 for i in range(n)] for j in range(n)]
        
    # Chow-Liu Algorithm
    def CL():
    

# Log Likelihood
def LL(p):
    return np.sum(np.log(p))

IndentationError: expected an indented block (<ipython-input-102-20a2a2df656e>, line 12)

In [106]:
test['accidents'][0]  == test['accidents'][1]

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True, False,  True, False,  True,  True, False,
        True, False,  True, False,  True, False, False,  True, False,
        True, False, False, False, False,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True, False,  True,
       False,  True, False,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [108]:
T = np.transpose(test['accidents'])

In [218]:
t = test['accidents']
d = np.transpose(train['accidents'])

#O(n^3)
def getWeights(T):
    d = np.transpose(T)
    n = len(d)
    matrix = [[0 for i in range(n)] for j in range(n)]
    for j in range(n):
        for i in range(n):
            if (i == j):
                matrix[j][i] = 0
            else:
                Ixy = 0
                for y in np.unique(d[i]):
                    for x in np.unique(d[j]):
                        # P(x,y) = P(x | y) * p(y)
                        X = d[i] == x
                        Y = d[j] == y
                        py = (sum(Y) + 1) / (len(Y) + n)
                        px = (sum(X) + 1) / (len(X) + n)
                        pxGy = (sum(X == Y) + 1) / (len(X) + 1)
                        pxy = pxGy * py
                        Ixy += pxy * np.log(pxy / (px * py))
                matrix[j][i] = Ixy
    return matrix

In [221]:
M = getWeights(train['accidents'][0:10])

In [266]:
minimum_spanning_tree(csr_matrix([[(i * -1) - 1  for i in m] for m in M])).

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [264]:
csr_matrix([[i * -1 for i in m] for m in M]).toarray()

array([[ 0.        , -0.24932055, -0.24932055, ..., -0.07923715,
        -0.04446455, -0.04446455],
       [-0.24932055,  0.        , -0.24932055, ..., -0.07923715,
        -0.04446455, -0.04446455],
       [-0.24932055, -0.24932055,  0.        , ..., -0.07923715,
        -0.04446455, -0.04446455],
       ...,
       [-0.08187574, -0.08187574, -0.08187574, ...,  0.        ,
        -0.25084015, -0.25084015],
       [-0.04446455, -0.04446455, -0.04446455, ..., -0.22811581,
         0.        , -0.24932055],
       [-0.04446455, -0.04446455, -0.04446455, ..., -0.22811581,
        -0.24932055,  0.        ]])

-0.24932054526026956